# Regression model in Keras
## A - Build a baseline model

### Import libraries

In [2]:
import pandas as pd
import numpy as np

### Load data

In [3]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Check dataset

In [4]:
concrete_data.shape

(1030, 9)

In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Sanity check

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Number of independent variables

In [9]:
n_cols = predictors.shape[1]

Randomly split the data into a training and test sets by holding 30% of the data for testing

### Additional step for B: normalize data

In [27]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)
print('Training set shapes: {} {}'.format(X_train.shape, y_train.shape))
print('Testing set shapes: {} {}'.format(X_test.shape, y_test.shape))

Training set shapes: (721, 8) (721,)
Testing set shapes: (309, 8) (309,)


### Build a neural network

Characteristics:

* One hidden layer of 10 nodes, and a ReLU activation function
* Use the **adam** optimizer and the **mean squared error**  as the loss function

In [29]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Define regression model

In [30]:
def baseline_regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Compile the model

In [31]:
model = baseline_regression_model()

### Train the model on the normalized training data using 50 epochs

In [32]:
model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=1)

Train on 504 samples, validate on 217 samples
Epoch 1/50
504/504 [==============================] - 3s 7ms/step - loss: 1714.0709 - val_loss: 1579.1366
Epoch 2/50
504/504 [==============================] - 0s 515us/step - loss: 1700.2464 - val_loss: 1567.5343
Epoch 3/50
504/504 [==============================] - 0s 550us/step - loss: 1687.4342 - val_loss: 1556.1353
Epoch 4/50
504/504 [==============================] - 0s 484us/step - loss: 1674.4708 - val_loss: 1545.3319
Epoch 5/50
504/504 [==============================] - 0s 490us/step - loss: 1662.2581 - val_loss: 1534.6414
Epoch 6/50
504/504 [==============================] - 1s 1ms/step - loss: 1650.4118 - val_loss: 1524.0501
Epoch 7/50
504/504 [==============================] - 0s 509us/step - loss: 1638.5095 - val_loss: 1513.8472
Epoch 8/50
504/504 [==============================] - 0s 477us/step - loss: 1627.2487 - val_loss: 1503.6790
Epoch 9/50
504/504 [==============================] - 1s 1000us/step - loss: 1615.7989 - val_l

### Evaluate the model on the normalized test data
Compute the mean squared error between the predicted concrete strength and the actual concrete strength

In [33]:
from sklearn.metrics import mean_squared_error
yhat = model.predict(X_test)
print("MSE: %.4f" % mean_squared_error(y_test , yhat))

MSE: 930.0869


### Repeat splitting, training and evaluation 50 times

Create a list of 50 MSE (don't forget to normalize)

In [34]:
mse_list = list()

for i in range(50):
    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)
    # Compile and train the model
    model = baseline_regression_model()
    model.fit(X_train, y_train, validation_split=0.3, epochs=50, verbose=0)
    # Predict based on test set and calculate MSE
    yhat = model.predict(X_test)
    mse = mean_squared_error(y_test , yhat)
    mse_list.append(mse)
    print("Iteration #%d --- MSE: %.4f" % (i+1, mse))

Iteration #1 --- MSE: 788.5825
Iteration #2 --- MSE: 575.5257
Iteration #3 --- MSE: 584.6038
Iteration #4 --- MSE: 622.8724
Iteration #5 --- MSE: 540.8183
Iteration #6 --- MSE: 628.3225
Iteration #7 --- MSE: 444.8380
Iteration #8 --- MSE: 622.6691
Iteration #9 --- MSE: 735.0844
Iteration #10 --- MSE: 709.2830
Iteration #11 --- MSE: 515.2374
Iteration #12 --- MSE: 601.6213
Iteration #13 --- MSE: 696.4471
Iteration #14 --- MSE: 582.7383
Iteration #15 --- MSE: 790.4270
Iteration #16 --- MSE: 714.3932
Iteration #17 --- MSE: 601.6921
Iteration #18 --- MSE: 649.0747
Iteration #19 --- MSE: 686.1694
Iteration #20 --- MSE: 628.1253
Iteration #21 --- MSE: 779.2245
Iteration #22 --- MSE: 721.1636
Iteration #23 --- MSE: 639.0287
Iteration #24 --- MSE: 743.1776
Iteration #25 --- MSE: 690.6295
Iteration #26 --- MSE: 671.2903
Iteration #27 --- MSE: 825.9683
Iteration #28 --- MSE: 537.9190
Iteration #29 --- MSE: 502.3863
Iteration #30 --- MSE: 552.5596
Iteration #31 --- MSE: 610.7293
Iteration #32 ---

### Report the mean and the standard deviation of the mean squared errors

In [35]:
print("Mean of MSE: %.4f" % np.mean(mse_list))
print("Standard deviation of MSE: %.4f" % np.std(mse_list))

Mean of MSE: 652.7311
Standard deviation of MSE: 120.2288


The mean of MSE for normalized data (652.7311) was higher than mean MSE from case A (334.7935)